In [ ]:
# Suppress TensorFlow and CUDA warnings
import os
import logging
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.getLogger('tensorflow').setLevel(logging.FATAL)
tf.get_logger().setLevel('ERROR')
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)


In [ ]:
# Import libraries
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt


In [ ]:
# Function to load images from a folder (limited to 100 images)
def load_images_from_folder(folder, max_images=100):
    images = []
    labels = []
    count = 0
    print(f"Loading images from {folder}...")
    for subdir, _, files in os.walk(folder):
        for filename in files:
            if filename.endswith(('.jpg', '.jpeg', '.png')) and count < max_images:
                label = 1 if 'occupied' in filename else 0
                img = cv2.imread(os.path.join(subdir, filename))
                if img is not None:
                    img = cv2.resize(img, (64, 64))
                    images.append(img)
                    labels.append(label)
                    count += 1
            if count >= max_images:
                break
    print(f"Total images loaded: {len(images)}")
    return np.array(images), np.array(labels)

# Replace with the path to your dataset
dataset_path = '/kaggle/input/pklot-dataset'
images, labels = load_images_from_folder(dataset_path, max_images=100)

# Normalize the images
images = images / 255.0

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [ ]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32)


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')

# Plot training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()


In [ ]:
# Save the model
model.save('parking_occupancy_model.h5')


In [ ]:
# Load and use the model for prediction
def predict_parking_occupancy(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to read image at {image_path}")
        return "Invalid Image"
    img = cv2.resize(img, (64, 64))
    img = np.expand_dims(img, axis=0) / 255.0
    prediction = model.predict(img)
    return 'Occupied' if prediction > 0.5 else 'Empty'

# Example usage with a valid image path
valid_image_path = '/kaggle/input/pklot-dataset/valid/2012-12-22_19_20_16_jpg.rf.5488350455fd1ae49f8864d2027e4161.jpg'
print(predict_parking_occupancy(valid_image_path))
